In [1]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import os
import findFiles
from findFiles import *
import comFuncs
import comFuncs3
import string
import rarfile
import re
from multiprocessing import Pool
import time
import resource
import functools
import more_itertools

In [2]:
t0=time.time()
try:
    res=comFuncs.load_obj("datas/indexR")
except:
    res=[]
    path='/media/some/sam/dall/'
    pattern='index*2016*.txt'
    names=findFiles(path,pattern)
    names=names+["/media/some/sam/dall/New ebook packs for August 2016/New ebook packs INDEX 2003-2015.txt"]
    res= [l.strip().split("\\")  for n in names for l in open(n).readlines() if l.count("\\")>0 ]
    comFuncs.save_obj(res,"datas/indexR")
print "time: ",time.strftime('%H:%M:%S', time.gmtime(time.time()-t0)),time.time()-t0
len(res),res[0]

time:  00:00:26 26.4569690228


(573313,
 ['04-01-2016 Part 1-2',
  'Adele Abbott - [The Witch PI 09] - Witch is When My Heart Broke (retail) (azw3).rar'])

In [3]:
def clean(indata):
    
    indata=re.sub("(\s*\(\s*[a-zA-Z0-9 ]*\)\s*)*.rar"," ",indata).strip()
    indata=re.sub("\([vV][0-9]\..+"," ",indata).strip()
    indata=re.sub("\s*\[(\,*\s*[A-Z]+-[0-9]+\,*\s*)+\]*\s*"," ",indata).strip()
    
    indata=indata.replace("'","")
    indata=re.sub('\W',' ',indata)
    indata=re.sub("\s+"," ",indata).strip()
    words=indata.lower().split()
    st=[WordNetLemmatizer().lemmatize(w) for w in words]
    stmw=list(more_itertools.unique_everseen(st))
    return stmw

In [4]:
def similarity(a,b):
    return comFuncs3.get_cosine(comFuncs3.text_to_vector(a.lower()),comFuncs3.text_to_vector(b.lower()))

In [5]:
def func(x):
    return [x[0],' '.join((clean(x[1]))),x[1]]

In [6]:

t0=time.time()
try:
    indd=comFuncs.load_obj("datas/inddBss")
except:
    p = Pool(10)
    indd=p.map(func,res[:10])
    p.terminate()
    p.close()
    comFuncs.save_obj(indd,"datas/inddBss")
print time.strftime('%H:%M:%S', time.gmtime(time.time()-t0)),time.time()-t0

00:00:31 31.7563860416


In [7]:
from itertools import izip, chain

def grouper(n, iterable):
    return zip(*[chain(iterable)]*n)

In [8]:
from multiprocessing import Process
import numpy

def do_work(data):

    return map(func,data)


In [9]:
t0=time.time()
try:
    indd=comFuncs.load_obj("datas/inddB")
except:
    indata=res
    num=8
    indt=list(grouper(num,indata))
    #indt[-1]=list(filter(None,indt[-1]))
    p = Pool(num)
    ress=p.map(do_work,indt)
    p.terminate()
    p.close()
    indd=list(chain(*ress))
    comFuncs.save_obj(indd,"datas/inddB")
print "time: ",time.strftime('%H:%M:%S', time.gmtime(time.time()-t0)),time.time()-t0
print "number of rec: ",len(indd)
print "first rec: ",indd[0]

time:  00:00:43 43.1650598049
number of rec:  573312
first rec:  ['04-01-2016 Part 1-2', 'adele abbott the witch pi 09 is when my heart broke', 'Adele Abbott - [The Witch PI 09] - Witch is When My Heart Broke (retail) (azw3).rar']


In [10]:
filter(lambda x:x[2].count("v5.0")>1 ,indd)

[['New Books 06-20-2009',
  'jane green second chance',
  'Jane Green - Second Chance (v5.0) (v5.0) (pdf).rar'],
 ['01-22-2011',
  u'john updike my father tear and other story',
  "John Updike - My Father's Tears and Other Stories (v5.0) (epub) (v5.0) (epub).rar"],
 ['01-27-2011',
  u'michael scott the secret of immortal nicholas flamel 01 alchemyst',
  'Michael Scott - [The Secrets of the Immortal Nicholas Flamel 01] - The Alchemyst (v5.0) (epub) (v5.0) (epub).rar'],
 ['03-04-2011',
  'charles cumming the trinity six 2011',
  'Charles Cumming - The Trinity Six (2011) (v5.0a) (epub) (v5.0) (epub).rar'],
 ['06-21-2014',
  'magazine clarkesworld 093 v5 0',
  'Magazine - Clarkesworld - 093 [v5.0] (v5.0) (epub).rar']]

In [11]:
#find files epub
dr='/media/some/sam/download-ep/'
try:
    lst=comFuncs.load_obj("datas/lsepubdown")
except:
    lst=findFiles(dr,"*.epub")

    comFuncs.save_obj(lst,"datas/lsepubdown")
len(lst)

4632

In [12]:
def searchindd(seTerm):
    
    if len(seTerm)>=1 and len(seTerm)<10:
        print "Search Term: ",' '.join(seTerm)
        allres=[ list
                (filter
                 (lambda x: 
                  x[1].count(se.strip().lower()) > 0, indd)) for se in seTerm if len(se)>2]
        #find common items in multiple lists
        return reduce(lambda sx, sy: filter(lambda x: x in sx,sy), ((i) for i in allres))
    else:
        return []

In [13]:
def findFile(f):
    fp,fn=os.path.split(f)
    ext=re.findall('.+(\..+)',fn)[0]
    se=clean(fn.replace(ext,""))
    try:
        return f,zip(*searchindd(se))[1]
    except:
        return f,[]

In [37]:
searchItem=raw_input("Enter Search term: ")
t0=time.time()
f=clean(searchItem)
sor=searchindd(f)
print "time: ",time.time()-t0
print "search: ",f
print "num of rec: ",len(sor)
if len(sor)>0:
    print "items:\n",'\n'.join(str(r)+"- "+zip(*sor)[1][r-1] for r in range(1,len(zip(*sor)[2])+1))

Enter Search term: hunger games
Search Term:  hunger game
time:  1.4015891552
search:  ['hunger', u'game']
num of rec:  47
items:
1- suzanne collins the hunger game
2- suzanne collins hunger game 02 catching fire
3- suzanne collins hunger game 02 catching fire
4- suzanne collins hunger game 02 catching fire
5- suzanne collins the hunger game v2 html
6- collins suzanne hunger game 02 catching fire
7- suzanne collins hunger game 01 the
8- suzanne collins hunger game 01 the
9- suzanne collins hunger game 01 the
10- suzanne collins hunger game 01 the
11- suzanne collins hunger game 01 the v2 0
12- suzanne collins hunger game 01 the v2 0
13- collins suzanne hunger game 03 mockingjay
14- collins suzanne hunger game 03 mockingjay
15- collins suzanne hunger game 03 mockingjay
16- collins suzanne hunger game 03 mockingjay
17- collins suzanne hunger game 03 mockingjay
18- collins suzanne hunger game 03 mockingjay
19- suzanne collins the hunger game 03 mockingjay html jpeg zip
20- suzanne collins

In [15]:
ll=list(filter(lambda x:x[2].lower().count("kenyon")>0 and x[2].lower().count("dark")>0 and \
               x[2].lower().count("v2.0")>0,indd))
if len(ll)>0:
    l=ll[0]
    print l[2]
    print clean(l[2])

Sherrilyn Kenyon - [Dark Hunter 02] - Night Embrace (V2.0-R2.rar
['sherrilyn', 'kenyon', 'dark', 'hunter', '02', 'night', 'embrace']


In [16]:
len(indd)

573312

In [21]:
def findr(name, path):
    #print name,path
    res=!find $path -type f -iname $name
    return res

In [38]:
foo="/home/some/hunger/"
!mkdir "$foo"
cnt=0
sorr=list(filter(lambda x: x[2].lower().count('epub') > 0, sor))+\
list(filter(lambda x: x[2].lower().count('epub') == 0, sor))

books=[]
for r in sorr:
 
        found=''.join(rr for rr in r[0] if rr in string.digits+"- ")
        baseName=r[1]
        if any(similarity(baseName,b.lower())>.75 for b in books):
            continue
        found=found.replace("  "," ").replace(" ","*").replace("_","*")+"*.rar"
        fls=findr(found,'/media/some/sam/dall/')
        #print found
        if len(fls)>0:
            for flss in fls:
                if re.match(".+.part\d+.rar",flss):
                    flss= re.sub(r".part\d+.rar", ".part1.rar", flss)
                    fls=filter(lambda x: x.count(flss.split(".part")[0])==0,fls)
                rf = rarfile.RarFile(flss)
                booFound=False
                for f in rf.infolist():
                    #print f.filename, f.file_size
                    if f.filename.count("\\")>0 and similarity(f.filename.split("\\")[1].replace(".rar",""),r[2])>0.9:
                        cnt+=1
                        booFound=True
                        print cnt,"- ",f.filename,baseName
                        try:
                            rf.extract(f,foo)
                            books.append(baseName)
                        except:
                            print "reading problem",r[1],""
                        ft=foo+f.filename.split('\\')[1]
                        fo=foo+f.filename.split('\\')[0]
                        fn=fo+"/"+f.filename.split('\\')[1]
                        os.rename(fn,ft) 
                        os.rmdir(fo)
                
                if booFound:
                    break
                                
            if not booFound:
                print r[2], "  not found"
                    
    
        else:
            print "couldnt find file  ",found
    

1 -  New Books 10-01-2009\Suzanne Collins - [Hunger Games 02] - Catching Fire (epub).rar suzanne collins hunger game 02 catching fire
2 -  New Books 08-07-2010\Suzanne Collins - [Hunger Games 01] - The Hunger Games (v2.0) (epub).rar suzanne collins hunger game 01 the
3 -  New Books 08-07-2010\Suzanne Collins - [Hunger Games 01] - The Hunger Games (v2.0) (lit).rar suzanne collins hunger game 01 the
4 -  New Books 08-07-2010\Suzanne Collins - [Hunger Games 01] - The Hunger Games (v2.0) (mobi).rar suzanne collins hunger game 01 the
5 -  New Books 08-07-2010\Suzanne Collins [Hunger Games 01] The Hunger Games v2.0 (ePub).rar suzanne collins hunger game 01 the
6 -  New Books 08-21-2010\Collins, Suzanne - Hunger Games 03 - Mockingjay (epub).rar collins suzanne hunger game 03 mockingjay
7 -  05-28-2011\Leah Wilson - The Girl Who Was on Fire- Your Favorite Authors on Suzanne Collins' Hunger Games Trilogy (epub).rar leah wilson the girl who wa on fire your favorite author suzanne collins hunger 

In [33]:
x="Nora Roberts - [Stanislaski 01] - Taming Natasha [ST-01] (v5.0) (epub).rar"
y="Nora Roberts - [Stanislaski 05] - Waiting for Nick [SSE-1088, MSE-1096,MSE-1096,DDD-2] (ve5.d0)(epub).rar"
re.sub("\s*\[[A-Z]+-[0-9]+(\,*\s*[A-Z]*-*[0-9]*)*\]\s*.+\(*[a-zA-Z]+[0-9]+\.*[0-9]*\)"," ",y)
re.sub("\s*\[(\,*\s*[A-Z]+-[0-9]+\,*\s*)+\]\s*(\(\s*.+\)\s*)*.rar"," ",y).strip()

'Nora Roberts - [Stanislaski 05] - Waiting for Nick'

In [ ]:
t0=time.time()

p = Pool(10)
flsss=p.map(findFile,lst[0:10])
p.terminate()
p.close()
print time.strftime('%H:%M:%S', time.gmtime(time.time()-t0)),time.time()-t0

In [ ]:
result=list(filter (lambda x: len(x[1])>0,flsss))
len(result)

In [ ]:
zip(*flsss)[0],zip(*flsss)[1]